<a href="https://colab.research.google.com/github/kingamirasadi/Python-Pytorch-sklearn/blob/main/fedFIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U "flwr[simulation]"


In [3]:
import flwr as fl
import ray
print("Flower:", fl.__version__)
print("Ray:", ray.__version__)


Flower: 1.22.0
Ray: 2.31.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
from google.colab import files

uploaded = files.upload()

Saving KDDTest+.txt to KDDTest+ (1).txt


In [62]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder , LabelEncoder , MinMaxScaler
from sklearn.model_selection import train_test_split
import torch
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score

import torch.nn as nn

from torch.utils.data import TensorDataset, DataLoader

from collections import OrderedDict

import flwr as fl


In [6]:
import torch.nn as nn
df = pd.read_csv('/content/KDDTest+.txt')


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [63]:
df.dtypes

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,0
0,int64
0.1,int64
0.2,int64
0.3,int64
0.4,int64
0.5,int64
0.6,int64
0.7,int64
0.8,int64
0.9,int64


In [8]:
df = df.drop('21' , axis =1 )

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [9]:
df.columns

Index(['0', 'tcp', 'private', 'REJ', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6',
       '0.7', '0.8', '0.9', '0.10', '0.11', '0.12', '0.13', '0.14', '0.15',
       '0.16', '0.17', '0.18', '229', '10', '0.00', '0.00.1', '1.00', '1.00.1',
       '0.04', '0.06', '0.00.2', '255', '10.1', '0.04.1', '0.06.1', '0.00.3',
       '0.00.4', '0.00.5', '0.00.6', '1.00.2', '1.00.3', 'neptune'],
      dtype='object')

In [10]:
df.dtypes

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,0
0,int64
tcp,object
private,object
REJ,object
0.1,int64
0.2,int64
0.3,int64
0.4,int64
0.5,int64
0.6,int64


In [11]:
df['tcp']

,tcp
0,tcp
1,tcp
2,icmp
3,tcp
4,tcp
...,...
22538,tcp
22539,tcp
22540,tcp
22541,udp


In [12]:
df['tcp'].nunique()

3

In [13]:
encoder = OneHotEncoder(sparse_output=False)

In [14]:
tcp_encoded = encoder.fit_transform(df[['tcp']])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [15]:
tcp_df = pd.DataFrame(tcp_encoded, columns=encoder.get_feature_names_out(['tcp']))

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [16]:
tcp_df

,tcp_icmp,tcp_tcp,tcp_udp
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
22538,0.0,1.0,0.0
22539,0.0,1.0,0.0
22540,0.0,1.0,0.0
22541,0.0,0.0,1.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [17]:
df = df.drop('tcp' , axis = 1)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [18]:
df = pd.concat([df, tcp_df], axis=1)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [19]:
df.sample(2)

,0,private,REJ,0.1,0.2,0.3,0.4,0.5,0.6,0.7,...,0.00.3,0.00.4,0.00.5,0.00.6,1.00.2,1.00.3,neptune,tcp_icmp,tcp_tcp,tcp_udp
17973,0,private,SF,1,1,0,0,0,0,0,...,0.97,0.00,0.0,0.0,0.03,0.0,satan,0.0,0.0,1.0
20118,0,http,SF,213,834,0,0,0,0,0,...,0.03,0.02,0.0,0.0,0.00,0.0,normal,0.0,1.0,0.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [20]:
xxxx = df['private'].value_counts()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [21]:
df['private'].value_counts()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,count
private,
http,7853
private,4773
telnet,1626
pop_3,1019
smtp,934
...,...
printer,11
ntp_u,10
rje,8


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [22]:
services =['http','private','telnet','telnet','smtp']

In [23]:
df['private'] = df['private'].where(df['private'].isin(services), 'other')

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [24]:
private_encoded =  encoder.fit_transform(df[['private']])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [25]:
df_encoded_private = pd.DataFrame(private_encoded, columns=encoder.get_feature_names_out(['private']))

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [26]:
df = df.drop('private' , axis =1)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [27]:
df = pd.concat([df, df_encoded_private], axis=1)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [28]:
df.dtypes

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,0
0,int64
REJ,object
0.1,int64
0.2,int64
0.3,int64
0.4,int64
0.5,int64
0.6,int64
0.7,int64
0.8,int64


In [29]:
df['REJ'].value_counts()


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,count
REJ,
SF,14875
REJ,3849
S0,2013
RSTO,773
RSTR,669
S3,249
SH,73
S1,21
S2,15


In [30]:
rej = ['SF','REJ','S0','RSTO','RSTR']

In [31]:
df['REJ'] = df['REJ'].where(df['REJ'].isin(rej), 'other')

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [32]:
rej_encoded = encoder.fit_transform(df[['REJ']])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [33]:
df_rej_encoded = pd.DataFrame(rej_encoded, columns=encoder.get_feature_names_out(['REJ']))

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [34]:
df_rej_encoded

,REJ_REJ,REJ_RSTO,REJ_RSTR,REJ_S0,REJ_SF,REJ_other
0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
22538,0.0,0.0,0.0,0.0,1.0,0.0
22539,0.0,0.0,0.0,0.0,1.0,0.0
22540,0.0,0.0,0.0,0.0,1.0,0.0
22541,0.0,0.0,0.0,0.0,1.0,0.0


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [35]:
df = df.drop('REJ' ,axis = 1)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [36]:
df = pd.concat([df, df_rej_encoded], axis=1)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [37]:
df['neptune'] = df['neptune'].replace(to_replace=[x for x in df['neptune'].unique() if x != 'normal'], value='attack')


In [38]:
label_encoder = LabelEncoder()

In [39]:
df['neptune'] = label_encoder.fit_transform(df['neptune'])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [40]:
df.dtypes

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,0
0,int64
0.1,int64
0.2,int64
0.3,int64
0.4,int64
0.5,int64
0.6,int64
0.7,int64
0.8,int64
0.9,int64


In [41]:
for col in df.columns:
    print(df[col].name , df[col].max(), df[col].min())

0 57715 0
0.1 62825648 0
0.2 1345927 0
0.3 1 0
0.4 3 0
0.5 3 0
0.6 101 0
0.7 4 0
0.8 1 0
0.9 796 0
0.10 1 0
0.11 2 0
0.12 878 0
0.13 100 0
0.14 5 0
0.15 4 0
0.16 0 0
0.17 1 0
0.18 1 0
229 511 0
10 511 0
0.00 1.0 0.0
0.00.1 1.0 0.0
1.00 1.0 0.0
1.00.1 1.0 0.0
0.04 1.0 0.0
0.06 1.0 0.0
0.00.2 1.0 0.0
255 255 0
10.1 255 0
0.04.1 1.0 0.0
0.06.1 1.0 0.0
0.00.3 1.0 0.0
0.00.4 1.0 0.0
0.00.5 1.0 0.0
0.00.6 1.0 0.0
1.00.2 1.0 0.0
1.00.3 1.0 0.0
neptune 1 0
tcp_icmp 1.0 0.0
tcp_tcp 1.0 0.0
tcp_udp 1.0 0.0
private_http 1.0 0.0
private_other 1.0 0.0
private_private 1.0 0.0
private_smtp 1.0 0.0
private_telnet 1.0 0.0
REJ_REJ 1.0 0.0
REJ_RSTO 1.0 0.0
REJ_RSTR 1.0 0.0
REJ_S0 1.0 0.0
REJ_SF 1.0 0.0
REJ_other 1.0 0.0


In [42]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [43]:
scaled_data = scaler.fit_transform(df)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [44]:
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)

In [45]:
len(scaled_df.columns)

53

In [46]:

for col in scaled_df.columns:
    print(scaled_df[col].name , scaled_df[col].max(), scaled_df[col].min())

0 1.0 0.0
0.1 1.0 0.0
0.2 1.0 0.0
0.3 1.0 0.0
0.4 1.0 0.0
0.5 1.0 0.0
0.6 1.0 0.0
0.7 1.0 0.0
0.8 1.0 0.0
0.9 1.0 0.0
0.10 1.0 0.0
0.11 1.0 0.0
0.12 1.0 0.0
0.13 1.0 0.0
0.14 1.0 0.0
0.15 1.0 0.0
0.16 0.0 0.0
0.17 1.0 0.0
0.18 1.0 0.0
229 1.0 0.0
10 1.0 0.0
0.00 1.0 0.0
0.00.1 1.0 0.0
1.00 1.0 0.0
1.00.1 1.0 0.0
0.04 1.0 0.0
0.06 1.0 0.0
0.00.2 1.0 0.0
255 1.0 0.0
10.1 1.0 0.0
0.04.1 1.0 0.0
0.06.1 1.0 0.0
0.00.3 1.0 0.0
0.00.4 1.0 0.0
0.00.5 1.0 0.0
0.00.6 1.0 0.0
1.00.2 1.0 0.0
1.00.3 1.0 0.0
neptune 1.0 0.0
tcp_icmp 1.0 0.0
tcp_tcp 1.0 0.0
tcp_udp 1.0 0.0
private_http 1.0 0.0
private_other 1.0 0.0
private_private 1.0 0.0
private_smtp 1.0 0.0
private_telnet 1.0 0.0
REJ_REJ 1.0 0.0
REJ_RSTO 1.0 0.0
REJ_RSTR 1.0 0.0
REJ_S0 1.0 0.0
REJ_SF 1.0 0.0
REJ_other 1.0 0.0


In [64]:
y = scaled_df['neptune'].to_numpy()

In [65]:
X = scaled_df.drop('neptune', axis=1).to_numpy()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [79]:
y.shape

(22543,)

In [80]:
X.shape

(22543, 52)

In [87]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=42)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [86]:
X_tr.shape[0]+X_test.shape[0]

22543

In [89]:
tensor_X_tr = torch.tensor(X_tr, dtype=torch.float32)
tensor_X_test = torch.tensor(X_test, dtype=torch.float32)
tensor_y_tr = torch.tensor(y_tr, dtype=torch.long)
tensor_y_test = torch.tensor(y_test, dtype=torch.long)

In [90]:
X_tr, X_val, y_tr, y_val = train_test_split(X_tr, y_tr, test_size=0.5/0.85, stratify=y_tr, random_state=42)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [91]:
train_ds = TensorDataset(torch.FloatTensor(X_tr), torch.LongTensor(y_tr))

In [92]:
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)

In [56]:
!pip install -U ray

  Using cached ray-2.49.2-cp312-cp312-manylinux2014_x86_64.whl.metadata (21 kB)
Using cached ray-2.49.2-cp312-cp312-manylinux2014_x86_64.whl (70.1 MB)
  Attempting uninstall: ray
    Found existing installation: ray 2.31.0
    Uninstalling ray-2.31.0:
      Successfully uninstalled ray-2.31.0


In [95]:
tensor_X_test.shape[0]+tensor_X_tr.shape[0]

22543

In [103]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from collections import OrderedDict
import numpy as np
import copy
from sklearn.metrics import precision_score, recall_score, f1_score

# -----------------------------
# Model definition
# -----------------------------
class FeatureEmbedding(nn.Module):
    def __init__(self, input_dim):
        super(FeatureEmbedding, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)

    def forward(self, x):
        # x shape: (batch, input_dim)
        x = x.unsqueeze(1)  # (batch, 1, input_dim)
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))  # (batch, 64, input_dim//4)
        return x


class AttentionBlock(nn.Module):
    def __init__(self, embed_dim=64, num_heads=8):
        super(AttentionBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
        self.norm = nn.BatchNorm1d(embed_dim)
        self.linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        attn_out, _ = self.attention(x, x, x)
        out = attn_out + x  # residual connection

        # BatchNorm along embedding dim
        out = out.permute(0, 2, 1)     # (batch, embed_dim, seq_len)
        out = self.norm(out)
        out = out.permute(0, 2, 1)     # back to (batch, seq_len, embed_dim)

        out = self.linear(out)
        return out


class FetFIDS(nn.Module):
    def __init__(self, input_dim, num_classes=5, num_heads=8, num_blocks=3):
        super(FetFIDS, self).__init__()
        self.input_dim = input_dim
        self.feature_embed = FeatureEmbedding(input_dim)

        # Attention blocks
        self.attention_blocks = nn.ModuleList([
            AttentionBlock(embed_dim=64, num_heads=num_heads)
            for _ in range(num_blocks)
        ])

        # Classifier head (+1 for raw input token concatenation)
        self.fc1 = nn.Linear((input_dim // 4 + 1) * 64, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.norm = nn.BatchNorm1d((input_dim // 4 + 1) * 64)

    def forward(self, x):
        # --- Feature Embedding ---
        feat = self.feature_embed(x)   # (batch, 64, input_dim//4)
        feat = feat.permute(0, 2, 1)   # (batch, input_dim//4, 64)

        # --- Raw Input as Extra Token ---
        raw = x.unsqueeze(-1)          # (batch, input_dim, 1)
        raw = F.interpolate(raw, size=64, mode='linear', align_corners=False)
        raw = raw.mean(dim=1, keepdim=True)  # (batch, 1, 64)

        # --- Combine ---
        combined = torch.cat([raw, feat], dim=1)  # (batch, input_dim//4+1, 64)

        # --- Attention Blocks ---
        for block in self.attention_blocks:
            combined = block(combined)

        # --- Flatten + Classifier ---
        combined = combined.reshape(combined.size(0), -1)
        combined = self.norm(combined)
        combined = F.relu(self.fc1(combined))
        return self.fc2(combined)


# -----------------------------
# Helper functions
# -----------------------------
def get_parameters(model):
    return [val.cpu().detach().numpy() for _, val in model.state_dict().items()]

def set_parameters(model, parameters):
    state_dict = model.state_dict()
    new_state = OrderedDict()
    for (k, v), p in zip(state_dict.items(), parameters):
        new_state[k] = torch.tensor(p, dtype=v.dtype)
    model.load_state_dict(new_state, strict=True)

def average_parameters(param_list, sizes):
    total = sum(sizes)
    new_params = []
    for params in zip(*param_list):
        avg = np.zeros_like(params[0], dtype=np.float64)
        for p, size in zip(params, sizes):
            avg += p * (size / total)
        new_params.append(avg)
    return new_params

def local_train(model, train_loader, device, epochs=20, lr=0.001, gamma=0.7):
    model = copy.deepcopy(model)
    model.to(device)
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

    for _ in range(epochs):
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
        scheduler.step()
    return model

def evaluate(model, test_loader, device):
    model.to(device)
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0
    criterion = nn.CrossEntropyLoss()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            preds = out.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
            loss_sum += loss.item() * yb.size(0)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(yb.cpu().numpy())
    accuracy = correct / total
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return loss_sum / total, accuracy, precision, recall, f1


# -----------------------------
# FedAvg Simulation
# -----------------------------
def fedavg_simulation(tensor_X_tr, tensor_y_tr, tensor_X_test, tensor_y_test,
                      num_classes=5, num_clients=5, num_rounds=20, local_epochs=20, batch_size=32):

    # Partition data for each client
    shard_size = len(tensor_X_tr) // num_clients
    client_data = []
    for i in range(num_clients):
        start, end = i * shard_size, (i + 1) * shard_size
        Xc, yc = tensor_X_tr[start:end], tensor_y_tr[start:end]
        client_data.append(TensorDataset(Xc, yc))

    # Test data shared across all clients
    test_loader = DataLoader(TensorDataset(tensor_X_test, tensor_y_test), batch_size=64)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Initialize global model
    global_model = FetFIDS(input_dim=tensor_X_tr.shape[1], num_classes=num_classes).to(device)

    for rnd in range(1, num_rounds + 1):
        print(f"\n--- Round {rnd} ---")
        local_params, local_sizes = [], []

        # Train each client locally
        for i, dataset in enumerate(client_data):
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            local_model = local_train(global_model, loader, device, epochs=local_epochs)
            params = get_parameters(local_model)
            local_params.append(params)
            local_sizes.append(len(dataset))
            print(f" Client {i} trained on {len(dataset)} samples")

        # FedAvg aggregation
        new_params = average_parameters(local_params, local_sizes)
        set_parameters(global_model, new_params)

        # Evaluate global model
        loss, accuracy, precision, recall, f1 = evaluate(global_model, test_loader, device)
        print(f" Global model after round {rnd}: loss={loss:.4f}, acc={accuracy*100:.2f}%, "
              f"precision={precision:.4f}, recall={recall:.4f}, f1={f1:.4f}")

    return global_model


# -----------------------------
# Main (use your prepared tensors)
# -----------------------------
if __name__ == "__main__":
    # Assuming tensor_X_tr, tensor_y_tr, tensor_X_test, tensor_y_test are defined
    num_classes = len(torch.unique(tensor_y_tr))

    fedavg_simulation(
        tensor_X_tr, tensor_y_tr,
        tensor_X_test, tensor_y_test,
        num_classes=num_classes,
        num_clients=5,
        num_rounds=20,
        local_epochs=20
    )


Using device: cuda

--- Round 1 ---
 Client 0 trained on 3832 samples
 Client 1 trained on 3832 samples
 Client 2 trained on 3832 samples
 Client 3 trained on 3832 samples
 Client 4 trained on 3832 samples
 Global model after round 1: loss=0.2954, acc=84.18%, precision=0.8673, recall=0.8418, f1=0.8350

--- Round 2 ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


 Client 0 trained on 3832 samples
 Client 1 trained on 3832 samples
 Client 2 trained on 3832 samples
 Client 3 trained on 3832 samples
 Client 4 trained on 3832 samples
 Global model after round 2: loss=0.4292, acc=80.01%, precision=0.8318, recall=0.8001, f1=0.7888

--- Round 3 ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


 Client 0 trained on 3832 samples
 Client 1 trained on 3832 samples
 Client 2 trained on 3832 samples
 Client 3 trained on 3832 samples
 Client 4 trained on 3832 samples
 Global model after round 3: loss=0.8333, acc=72.21%, precision=0.7774, recall=0.7221, f1=0.6919

--- Round 4 ---


...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


 Client 0 trained on 3832 samples
 Client 1 trained on 3832 samples
 Client 2 trained on 3832 samples
 Client 3 trained on 3832 samples
 Client 4 trained on 3832 samples
 Global model after round 19: loss=0.0602, acc=97.78%, precision=0.9779, recall=0.9778, f1=0.9778

--- Round 20 ---


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


 Client 0 trained on 3832 samples
 Client 1 trained on 3832 samples
 Client 2 trained on 3832 samples
 Client 3 trained on 3832 samples
 Client 4 trained on 3832 samples
 Global model after round 20: loss=0.0572, acc=98.05%, precision=0.9806, recall=0.9805, f1=0.9805


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
